In [12]:
import csv
import random
from torchtext.data import get_tokenizer
import torchtext

import torch
import torch.optim as optim
import torch.nn as nn 
import torch.nn.functional as F 

from tqdm import tqdm

import random

# import xlsxwriter module
import xlsxwriter

import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [19]:
workbook = xlsxwriter.Workbook('kawin.xlsx')
 
worksheet = workbook.add_worksheet()


In [20]:
TEST_DATA_PATH  = './EnglishDictionary.csv'
TOP_K           = 1
MODEL_NAME      = "./3dbert_mask_lr2_more.pt"

dataset = []
with open(TEST_DATA_PATH) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        dataset.append((row[3], row[0]))

# TODO: Check if the first row is header or actual values

dataset = dataset[1:]

print(len(dataset))

176009


In [21]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

In [22]:
en_de = open("en-de.txt", "r").readlines()

en_de_dict = {}

for i in en_de:
    en, de = i.split(" ")
    if en in en_de_dict:
        en_de_dict[en] = en_de_dict[en] + [de]
    else:
        en_de_dict[en] = [de]

In [23]:
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
device = torch.device("cuda:2")

model.load_state_dict(torch.load(MODEL_NAME))
model.to(device)

#loss_fn = F.cross_entropy
opt = optim.Adam(model.parameters(), lr = 0.0002)

In [24]:
for ip in range(len(dataset)):
    pred_words = []
    pred_word_english = []
    for j in range(1, 6):
        inputs = tokenizer("[MASK] "*j + dataset[ip][0], return_tensors="pt")

        inputs.to(device)
        with torch.no_grad():
            logits = model(**inputs).logits

        # retrieve index of [MASK]
        mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

        predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
        pred = tokenizer.decode(predicted_token_id)

        if len(pred.split(" ")) == 1:
            if pred in en_de_dict:
                for predictions in en_de_dict[pred]:
                    if predictions not in pred_words:
                        pred_words.append(predictions[:-1])
        for english_word in pred.split(" "):
            if english_word not in pred_word_english:
                pred_word_english.append(english_word)
    print(ip, pred_words)
    print(pred_word_english)
    if len(pred_words) >= TOP_K:
        if TOP_K == 1:
            worksheet.write('A'+str(ip+1), pred_words[0])
    else:
        if TOP_K == 1:
            ii = 0
            while '#' in pred_word_english[ii]:
                ii += 1
            worksheet.write('A'+str(ip+1), pred_word_english[ii])
workbook.close()

0 ['ein', 'eine', 'einen', 'einem', 'ein', 'eine', 'aleph']
['a', 'aleph', 'alephph', 'phoenil', 'phphthhtph']
1 ['ein', 'eine', 'einen', 'einem', 'ein', 'eine', 'sehen', 'schau', 'see', 'siehe', 'ansehen']
['a', 'see', 'se', '-a', 'me', '-', '-e']
2 ['adjektiv']
['adjective', 'ul', '-', 'idmina', 'idimina', 'heteroolica']
3 ['nam']
['nam', 'twentyoat', 'twenty', '-n', 'su', '-', 'an', 'ci']
4 ['eine', 'ein', 'ein', 'eine', 'einer', 'einen', 'einem', 'ann']
['an', 'ann', 'ahiln', 'aa', "'n", "aa''e"]
5 ['ein', 'eine', 'einen', 'einem', 'ein', 'eine']
['a', 'preps', 'prompts', 'antapsits', 'anta', '-sits']
6 ['ein', 'eine', 'einen', 'einem', 'ein', 'eine', 'eine', 'ein', 'ein', 'eine', 'einer', 'einen', 'einem']
['a', 'an', "'n", "a''n", "a''", '-e']
7 ['hase']
['ye', 'hase', "'a", "he''h", "t''", "'h"]
8 ['null', 'spe']
['null', 'spe', 'explee', 'anaspplee', 'anaphphplee']
9 ['ein', 'eine', 'einen', 'einem', 'ein', 'eine']
['a', 'ab', '-', "ana'-", "ab''", "ab'''-"]
10 ['grade', 'klass

In [ ]:
in lr 1 - ep 4 11.43
lr 2 ep 2 11.18
more 3 1063